In [1]:
import numpy as np
import pandas as pd
df = pd.read_excel('GE.xlsx')

In [2]:
course_list = df.iloc[:, 2].tolist()

In [3]:
course_list = course_list[3:]

In [4]:
course_list = list(set(course_list))

In [5]:
with open('GE course name.txt', 'w') as file:
    for course in course_list:
        file.write(str(course) + '\n')

In [6]:
course_list[:10]

['GEST1001',
 'GEST1017',
 'GELH2001',
 'GEST1019',
 'GELH2008',
 'GEST1015',
 'GEGA1000',
 'GESB2004',
 'GESB1003',
 'GELH1009']

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from tqdm import tqdm

courses = []
with open('GE course name.txt', 'r') as file:
    lines = file.read().splitlines()
    courses = [line.strip() for line in lines]

df = pd.DataFrame(columns=['course_name', 'name', 'overall_grade', 'grade', 'difficulty', 'useful', 'comments'])

wd = webdriver.Edge()
wd.implicitly_wait(2)

total_entries = 0

pbar = tqdm(total=len(courses), desc="Processing courses", ncols=100)

for course_name in courses:
    wd.get(f"https://www.umeh.top/course/{course_name}")
    time.sleep(1)
    
    try:
        teacher_blocks = WebDriverWait(wd, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="/reviews/"]'))
        )
        
        course_entries = 0
        for block in teacher_blocks:
            try:
                name = block.find_element(By.CSS_SELECTOR, 'div.break-words').text
                overall_grade_element = WebDriverWait(block, 2).until(
                    EC.presence_of_element_located((By.XPATH, './/div[@class="text-sm font-semibold"]/div[2]'))
                )
                overall_grade = overall_grade_element.text
                
                stats = block.find_elements(By.CSS_SELECTOR, 'div.flex.flex-row.text-xs.font-semibold.space-x-2 > div')
                grade = stats[0].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                difficulty = stats[1].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                useful = stats[2].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                comments = int(stats[3].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text)
                
                new_row = pd.Series([course_name, name, overall_grade, grade, difficulty, useful, comments], index=df.columns)
                df.loc[len(df)] = new_row
                
                total_entries += 1
                course_entries += 1
            
            except Exception as e:
                pbar.write(f"something went wrong with {course_name}")
                continue
        
            pbar.set_description(f"Processing {course_name}: {name} | Total entries: {total_entries}")
    
    except Exception as e:
        pbar.write(f"something went wrong with {course_name}")
        continue
    
    pbar.update(1)

wd.quit()
pbar.close()

Processing GESB1003: TERESA LANCRY DE GOUVEIA DE ALBUQUERQUE E SOUSA ROBALO | Total entries: 74:  16

something went wrong with GELH1009


Processing GEST2000: LEI SI MAN | Total entries: 92:  26%|██▌       | 15/58 [00:36<01:22,  1.91s/it]

something went wrong with GEGA1001


Processing GEST1005: CHAN WING HAN BRENDA | Total entries: 94:  29%|▎| 17/58 [00:43<01:44,  2.54s/it

...

Processing GESB2002: ZHANG LIWEN | Total entries: 250:  74%|█████▉  | 43/58 [01:44<00:48,  3.23s/it]

something went wrong with GESB1015


Processing GEST1010: MAK PENG UN | Total entries: 271:  86%|██████▉ | 50/58 [02:00<00:16,  2.01s/it]

something went wrong with GEGA2008


Processing GESB2003: MAC VAI IUN | Total entries: 273:  88%|███████ | 51/58 [02:04<00:17,  2.44s/it]


In [10]:
# save to csv
df.to_csv('GE_raw.csv', index=False)

In [11]:
df

,course_name,name,overall_grade,grade,difficulty,useful,comments
0,GEST1001,KUAN UN FONG,C+,B,B,B,40
1,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45
2,GEST1001,YEUNG YAU LUN,B-,C,C,C+,3
3,GEST1001,JIAO YIFAN,B-,B,B-,B,4
4,GEST1001,PUN SAU TAK,D,D,D+,D,18
...,...,...,...,...,...,...,...
268,GESB1005,KONG WA,N/A,N/A,N/A,N/A,0
269,GEST1010,IP WENG FAI,C,D+,C-,D+,15
270,GEST1010,MAK PENG UN,N/A,N/A,N/A,N/A,0
271,GESB2003,JOSEPH ADEA SY-CHANGCO,C,C+,C+,C-,20
